# imports

In [1]:
# Celda 1: Carga de Librerías y Datos
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import catboost as cb


In [2]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

global_mean = train_data['rating'].mean()
print(f"Media global de ratings: {global_mean:.4f}")

Media global de ratings: 7.6047


In [3]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [4]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [5]:
#train_data, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

In [6]:
# Obtener listas únicas de usuarios e ítems del conjunto de entrenamiento
unique_users = train_data['user'].unique()
unique_items = train_data['item'].unique()

# Crear índices para usuarios e ítems
user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

# Dimensiones de la matriz de ratings
num_users = len(unique_users)
num_items = len(unique_items)

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")

Número de usuarios: 73456
Número de ítems: 171171


In [7]:
from scipy.sparse import csr_matrix

# Crear la matriz dispersa solo con el conjunto de entrenamiento
rows = train_data['user'].map(user_to_index).values
cols = train_data['item'].map(item_to_index).values
ratings = train_data['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (73456, 171171)
Número de ratings no nulos en la matriz: 390351


# nmf

In [11]:
# Celda 3: Inicialización de Parámetros y Factores Latentes con Bias
# Hiperparámetros
k = 20  # Número de factores latentes
lambda_ = 0.015  # Regularización
learning_rate = 0.003  # Tasa de aprendizaje
num_epochs = 60  # Número de iteraciones

# Media global de ratings
mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Factores latentes inicializados: U ((73456, 20)), V ((171171, 20))
Sesgos inicializados: bu ((73456,)), bi ((171171,))


In [12]:
def train_nmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()  # Índices de las posiciones no nulas en la matriz
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]  # Rating real del usuario i sobre el ítem j

            # Predicción incluyendo el sesgo
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los factores latentes y sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Costo regularizado
            total_cost += error**2 + (lambda_ / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(V[j, :])**2 + bu[i]**2 + bi[j]**2)

        print(f"Época {epoch + 1}/{num_epochs} - Costo total: {total_cost:.4f}")

    return U, V, bu, bi


In [13]:
# Celda 5: Entrenamiento del Modelo NMF
print("Iniciando entrenamiento NMF...")
U, V, bu, bi = train_nmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")


Iniciando entrenamiento NMF...
Época 1/60 - Costo total: 1244721.9616
Época 2/60 - Costo total: 1167359.6045
Época 3/60 - Costo total: 1125079.1228
Época 4/60 - Costo total: 1094260.3815
Época 5/60 - Costo total: 1069459.4390
Época 6/60 - Costo total: 1048400.3122
Época 7/60 - Costo total: 1029879.3203
Época 8/60 - Costo total: 1013154.8572
Época 9/60 - Costo total: 997701.5720
Época 10/60 - Costo total: 983091.8908
Época 11/60 - Costo total: 968943.8066
Época 12/60 - Costo total: 954937.6545
Época 13/60 - Costo total: 940924.5068
Época 14/60 - Costo total: 927074.4054
Época 15/60 - Costo total: 913842.8955
Época 16/60 - Costo total: 901615.9509
Época 17/60 - Costo total: 890402.2016
Época 18/60 - Costo total: 879940.2094
Época 19/60 - Costo total: 869951.3420
Época 20/60 - Costo total: 860238.9406
Época 21/60 - Costo total: 850676.9110
Época 22/60 - Costo total: 841183.4039
Época 23/60 - Costo total: 831705.8556
Época 24/60 - Costo total: 822214.7271
Época 25/60 - Costo total: 812700.

# catboost

In [14]:
def build_dataset_for_xgboost(R_sparse, U, V):
    rows, cols = R_sparse.nonzero()
    X = []
    y = []

    for i, j in zip(rows, cols):
        u_vec = U[i]  # vector latente del usuario
        v_vec = V[j]  # vector latente del ítem

        x_input = np.concatenate([u_vec, v_vec])  # entrada al modelo
        rating = R_sparse[i, j]

        X.append(x_input)
        y.append(rating)

    return np.array(X), np.array(y)

In [16]:
# Crear datos
X, y = build_dataset_for_xgboost(R_sparse, U, V)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Crear y entrenar el modelo
cb_model = cb.CatBoostRegressor(loss_function='RMSE', n_estimators=200)
print("Iniciando entrenamiento CatBoost...")
cb_model.fit(X_train, y_train)

# Evaluar
y_pred = cb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE en test: {rmse:.4f}")


Iniciando entrenamiento CatBoost...
Learning rate set to 0.382527
0:	learn: 1.7912007	total: 167ms	remaining: 33.2s
1:	learn: 1.7399710	total: 192ms	remaining: 19s
2:	learn: 1.6927268	total: 219ms	remaining: 14.4s
3:	learn: 1.6517297	total: 244ms	remaining: 12s
4:	learn: 1.6327029	total: 268ms	remaining: 10.4s
5:	learn: 1.5944837	total: 295ms	remaining: 9.55s
6:	learn: 1.5584475	total: 321ms	remaining: 8.85s
7:	learn: 1.5414457	total: 346ms	remaining: 8.29s
8:	learn: 1.5097899	total: 372ms	remaining: 7.9s
9:	learn: 1.4815609	total: 400ms	remaining: 7.6s
10:	learn: 1.4595856	total: 424ms	remaining: 7.29s
11:	learn: 1.4358143	total: 449ms	remaining: 7.03s
12:	learn: 1.4244384	total: 474ms	remaining: 6.81s
13:	learn: 1.4040243	total: 497ms	remaining: 6.6s
14:	learn: 1.3930038	total: 521ms	remaining: 6.42s
15:	learn: 1.3740860	total: 546ms	remaining: 6.28s
16:	learn: 1.3625855	total: 569ms	remaining: 6.12s
17:	learn: 1.3446903	total: 596ms	remaining: 6.02s
18:	learn: 1.3280115	total: 622ms

In [19]:
def predict_rating(uid, iid, U, V, cb_model, user_to_index, item_to_index, mu):
    # Verifica si el usuario y el ítem existen en los índices
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]

        # Construimos el vector de entrada para XGBoost
        u_vec = U[user_idx]
        v_vec = V[item_idx]
        x_input = np.concatenate([u_vec, v_vec]).reshape(1, -1)

        # Predicción con XGBoost
        pred = cb_model.predict(x_input)[0]
    else:
        # Si no se puede predecir, devolver la media global
        pred = mu

    # Clamping entre 1 y 10 con formato decimal ".0"
    return f"{round(max(1, min(10, pred)))}.0"



# generacion csv

In [20]:
# Celda 10: Generación del Archivo de Predicciones
print("Generando predicciones finales...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid=uid, iid=iid, 
                   U=U, V=V, 
                   cb_model=cb_model, 
                   user_to_index=user_to_index, 
                   item_to_index=item_to_index, 
                   mu=mu)

    predictions.append((row_id, pred_rating))

# Crear el DataFrame con las predicciones
predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

# Guardar las predicciones en formato CSV
output_filename = "predictions_pmf_catboost.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado correctamente.")

Generando predicciones finales...
Archivo 'predictions_pmf_catboost.csv' generado correctamente.
